In [1]:
# from transformers import AutoModelForSequenceClassification
# from transformers import TFAutoModelForSequenceClassification
# from transformers import AutoTokenizer, AutoConfig
import numpy as np
# from scipy.special import softmax
import json
from tqdm import tqdm
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [5]:

# MODEL = r"cardiffnlp/twitter-xlm-roberta-base-sentiment"

# tokenizer = AutoTokenizer.from_pretrained(MODEL)
# config = AutoConfig.from_pretrained(MODEL)

# # # PT
# model = AutoModelForSequenceClassification.from_pretrained(MODEL).to('cuda:0')
model = SentimentIntensityAnalyzer()
read_file= open('Data/complete_comments.jsonl',encoding='utf-8')
write_file = open('Data/complete_comments_with_vader_2.jsonl','a',encoding='utf-8')

buffer = []
read_ids = set()

In [6]:
with open('Data/complete_comments_with_vader_2.jsonl','r') as file:
    for line in file:
        line=json.loads(line)
        read_ids.add(line['id'])

In [7]:
# try:
#     for line in tqdm(read_file):
#         line = json.loads(line)
#         if line['id'] in read_ids:
#             continue
#         read_ids.add(line['id'])

#         text = line['comment']
#         text = preprocess(text)
#         encoded_input = tokenizer(text, return_tensors='pt',truncation=True,max_length=512).to('cuda:0')
#         output = model(**encoded_input)
#         scores = output[0][0].cpu().detach().numpy()
#         scores = softmax(scores)
#         classification = {config.id2label[i]:float(scores[i]) for i in range(len(scores))}
#         buffer.append({
#             'id':line['id'],
#             'userId':line['userId'],
#             'videoId':line['videoId'],
#             'classification':classification
#         })
#         if len(buffer)>=1000:
#             for comment in buffer:
#                 json.dump(comment,write_file,ensure_ascii=False)
#                 write_file.write('\n')
#             buffer = []
# except KeyboardInterrupt:
#     write_file.close()
#     read_file.close()

In [ ]:
try:
    for line in tqdm(read_file):
        line = json.loads(line)
        if line['id'] in read_ids:
            continue
        read_ids.add(line['id'])

        text = line['comment']
        scores = model.polarity_scores(text)
        buffer.append({
            'id':line['id'],
            'userId':line['userId'],
            'videoId':line['videoId'],
            'category':line['category'],
            'publishedAt':line['publishedAt'],
            'classification':scores
        })
        if len(buffer)>=100000:
            for comment in buffer:
                json.dump(comment,write_file,ensure_ascii=False)
                write_file.write('\n')
            buffer = []
except KeyboardInterrupt:
    write_file.close()
    read_file.close()

25854096it [55:50, 5658.94it/s] 

In [ ]:
write_file.close()
read_file.close()